In [ ]:
#Prophet

In [1]:
!pip install prophet

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import os
from prophet import Prophet
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"  # Folder where crop-based CSVs are stored
output_models = "prophet_crop_models"
output_csv = "prophet_crop_csv"
output_metrics_csv = "prophet_crop_metrics.csv"
output_graphs = "prophet_crop_graphs"

# Create output folders if they don't exist
os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Function to parse dates safely
# -----------------------------
def parse_dates_safe(date_series):
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series, dayfirst=False)

# -----------------------------
# Function to calculate MAPE
# -----------------------------
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Avoid division by zero
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# -----------------------------
# Prepare metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each crop CSV
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"\n========== Processing Crop: {file} ==========")

        # Load CSV
        df = pd.read_csv(file_path)

        # -----------------------------
        # Parse and sort by date
        # -----------------------------
        df['Date'] = parse_dates_safe(df['Date'])
        df = df.sort_values('Date')

        # -----------------------------
        # Handle missing values in Average Price
        # -----------------------------
        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'].fillna(mean_value, inplace=True)
            print(f"Filled missing Average Price values with mean: {mean_value:.2f}")

        # Round prices for cleaner output
        df['Average Price'] = df['Average Price'].round(2)

        # -----------------------------
        # Prepare data for Prophet
        # -----------------------------
        prophet_df = df[['Date', 'Average Price']].rename(columns={'Date': 'ds', 'Average Price': 'y'})

        # -----------------------------
        # Build and fit Prophet model
        # -----------------------------
        model = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True)
        model.fit(prophet_df)

        # -----------------------------
        # Predict for all existing dates
        # -----------------------------
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)
        df['Predicted'] = forecast['yhat']

        # -----------------------------
        # Handle NaN values safely
        # -----------------------------
        df = df.dropna(subset=['Average Price', 'Predicted'])

        # -----------------------------
        # Rename for clarity
        # -----------------------------
        df.rename(columns={'Average Price': 'Actual'}, inplace=True)
        df['Predicted'] = df['Predicted'].round(2)

        # -----------------------------
        # Calculate Metrics
        # -----------------------------
        y_true = df['Actual'].values
        y_pred = df['Predicted'].values
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)
        mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
        accuracy = round(100 - mape, 2)

        print(f"📊 Metrics for {file}:")
        print(f"  MAE        : {mae}")
        print(f"  RMSE       : {rmse}")
        print(f"  R²         : {r2}")
        print(f"  MAPE(%)    : {mape}")
        print(f"  Accuracy(%) : {accuracy}\n")

        # -----------------------------
        # Save Metrics
        # -----------------------------
        metrics_list.append({
            'Crop File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # -----------------------------
        # Save Model
        # -----------------------------
        model_file = os.path.join(output_models, file.replace(".csv", "_prophet_crop_model.pkl"))
        joblib.dump(model, model_file)

        # -----------------------------
        # Save Updated CSV
        # -----------------------------
        save_df = df[['Date', 'Actual', 'Predicted']]
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_prophet_crop_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)
        print(f"💾 Saved updated CSV: {updated_csv_path}")

        # -----------------------------
        # Moving Averages for visualization
        # -----------------------------
        df['MA7'] = df['Actual'].rolling(7).mean()
        df['MA30'] = df['Actual'].rolling(30).mean()

        # -----------------------------
        # Plot Graph
        # -----------------------------
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['Actual'], label='Actual', color='blue')
        plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red')
        plt.plot(df['Date'], df['MA7'], label='MA7', color='green', linestyle='--')
        plt.plot(df['Date'], df['MA30'], label='MA30', color='orange', linestyle='--')
        plt.title(f"{file} - Actual vs Predicted (Crop-Based Forecast)")
        plt.xlabel("Date")
        plt.ylabel("Price (₹)")
        plt.legend()
        plt.grid(True)
        graph_path = os.path.join(output_graphs, file.replace(".csv", "_prophet_crop_graph.png"))
        plt.savefig(graph_path, bbox_inches='tight')
        plt.close()
        print(f"📈 Graph saved: {graph_path}")

# -----------------------------
# Save All Metrics to CSV
# -----------------------------
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)
print(f"\n✅ All crop-based datasets processed successfully!")
print(f"📊 Metrics saved to: {output_metrics_csv}")
print(f"📁 Models, updated CSVs, and graphs saved in their respective folders.")


========== Processing Crop: capsicum_2010_2024_master.csv ==========


21:32:40 - cmdstanpy - INFO - Chain [1] start processing
21:35:35 - cmdstanpy - INFO - Chain [1] done processing


📊 Metrics for capsicum_2010_2024_master.csv:
  MAE        : 923.59
  RMSE       : 1338.72
  R²         : -0.0674
  MAPE(%)    : 34.48
  Accuracy(%) : 65.52

💾 Saved updated CSV: prophet_crop_csv\capsicum_2010_2024_master_prophet_crop_updated.csv
📈 Graph saved: prophet_crop_graphs\capsicum_2010_2024_master_prophet_crop_graph.png

========== Processing Crop: onion_2010_2024_master.csv ==========


21:36:09 - cmdstanpy - INFO - Chain [1] start processing
21:47:05 - cmdstanpy - INFO - Chain [1] done processing


📊 Metrics for onion_2010_2024_master.csv:
  MAE        : 753.46
  RMSE       : 1085.48
  R²         : 0.0788
  MAPE(%)    : 79.16
  Accuracy(%) : 20.84

💾 Saved updated CSV: prophet_crop_csv\onion_2010_2024_master_prophet_crop_updated.csv
📈 Graph saved: prophet_crop_graphs\onion_2010_2024_master_prophet_crop_graph.png

========== Processing Crop: tomato_2010_2024_master.csv ==========


21:47:48 - cmdstanpy - INFO - Chain [1] start processing
21:58:24 - cmdstanpy - INFO - Chain [1] done processing


📊 Metrics for tomato_2010_2024_master.csv:
  MAE        : 636.64
  RMSE       : 899.94
  R²         : 0.1964
  MAPE(%)    : 78.86
  Accuracy(%) : 21.14

💾 Saved updated CSV: prophet_crop_csv\tomato_2010_2024_master_prophet_crop_updated.csv
📈 Graph saved: prophet_crop_graphs\tomato_2010_2024_master_prophet_crop_graph.png

========== Processing Crop: wheat_2010_2024_master.csv ==========


21:59:14 - cmdstanpy - INFO - Chain [1] start processing
22:03:01 - cmdstanpy - INFO - Chain [1] done processing


📊 Metrics for wheat_2010_2024_master.csv:
  MAE        : 297.88
  RMSE       : 473.69
  R²         : -0.3324
  MAPE(%)    : 17.39
  Accuracy(%) : 82.61

💾 Saved updated CSV: prophet_crop_csv\wheat_2010_2024_master_prophet_crop_updated.csv
📈 Graph saved: prophet_crop_graphs\wheat_2010_2024_master_prophet_crop_graph.png

✅ All crop-based datasets processed successfully!
📊 Metrics saved to: prophet_crop_metrics.csv
📁 Models, updated CSVs, and graphs saved in their respective folders.


In [ ]:
#ARIMA

In [3]:

import pandas as pd
import numpy as np
import os
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# =============================
# Paths
# =============================
input_folder = "dataset"  # Crop-based data folder
output_models = "arimax_crop_models"
output_csv = "arimax_crop_csv"
output_graphs = "arimax_crop_graphs"
output_metrics_csv = "arimax_crop_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# =============================
# Storage for metrics
# =============================
metrics_list = []

# =============================
# Safe Date Parsing
# =============================
def parse_dates_safe(date_series):
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series)

# =============================
# MAPE Calculation
# =============================
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# =============================
# Process Each Crop CSV
# =============================
print("\n🔄 Starting ARIMAX Crop Processing...\n")

for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        print(f"✅ Processing Crop: {file}")
        filepath = os.path.join(input_folder, file)

        # Load Data
        df = pd.read_csv(filepath)
        df['Date'] = parse_dates_safe(df['Date'])
        df.sort_values("Date", inplace=True)
        df.reset_index(drop=True, inplace=True)

        # Handle Missing Target
        df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)

        # =============================
        # Create Features
        # =============================
        df['Lag_1'] = df['Average Price'].shift(1)
        df['Lag_7'] = df['Average Price'].shift(7)
        df['MA_7'] = df['Average Price'].rolling(7).mean()
        df['MA_30'] = df['Average Price'].rolling(30).mean()

        df = df.dropna()  # Remove rows where lag/MA features unavailable

        # =============================
        # Train ARIMAX Model
        # =============================
        exog_features = ['Lag_1', 'Lag_7', 'MA_7', 'MA_30']

        model = SARIMAX(df['Average Price'], 
                        exog=df[exog_features],
                        order=(1,1,1))

        model_fit = model.fit(disp=False)

        # Predict
        df['Predicted'] = model_fit.predict(start=0, end=len(df)-1, exog=df[exog_features])
        df['Actual'] = df['Average Price']

        # =============================
        # Metrics Calculation
        # =============================
        y_true = df['Actual']
        y_pred = df['Predicted']

        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)
        mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
        accuracy = round(100 - mape, 2)

        print(f"📊 Metrics for {file}:")
        print(f" MAE   : {mae}")
        print(f" RMSE  : {rmse}")
        print(f" R²    : {r2}")
        print(f" MAPE  : {mape}%")
        print(f" ✅ Accuracy: {accuracy}%\n")

        # Save Metrics in List
        metrics_list.append({
            "Crop File": file,
            "MAE": mae,
            "RMSE": rmse,
            "R2": r2,
            "MAPE(%)": mape,
            "Accuracy(%)": accuracy
        })

        # =============================
        # Save Updated CSV
        # =============================
        save_df = df[['Date', 'Actual', 'Predicted']]
        csv_save_path = os.path.join(output_csv, file.replace(".csv", "_arimax_crop_updated.csv"))
        save_df.to_csv(csv_save_path, index=False)

        # =============================
        # Save Graph
        # =============================
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['Actual'], label='Actual', linewidth=2)
        plt.plot(df['Date'], df['Predicted'], label='Predicted', linewidth=2)
        plt.title(f"ARIMAX Forecast - {file}")
        plt.xlabel("Date")
        plt.ylabel("Price (₹)")
        plt.grid(True)
        plt.legend()

        graph_path = os.path.join(output_graphs, file.replace(".csv", "_arimax_crop_graph.png"))
        plt.savefig(graph_path, bbox_inches="tight")
        plt.close()

# =============================
# Save Metrics CSV
# =============================
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)

print("\n✅ ARIMAX Crop Forecasting Completed!")
print(f"📁 Models, CSV, Graphs saved successfully.")


🔄 Starting ARIMAX Crop Processing...

✅ Processing Crop: capsicum_2010_2024_master.csv
📊 Metrics for capsicum_2010_2024_master.csv:
 MAE   : 636.08
 RMSE  : 924.06
 R²    : 0.5984
 MAPE  : 30.95%
 ✅ Accuracy: 69.05%

✅ Processing Crop: onion_2010_2024_master.csv
📊 Metrics for onion_2010_2024_master.csv:
 MAE   : 389.83
 RMSE  : 636.25
 R²    : 0.6603
 MAPE  : 31.53%
 ✅ Accuracy: 68.47%

✅ Processing Crop: tomato_2010_2024_master.csv
📊 Metrics for tomato_2010_2024_master.csv:
 MAE   : 384.88
 RMSE  : 651.3
 R²    : 0.7019
 MAPE  : 42.55%
 ✅ Accuracy: 57.45%

✅ Processing Crop: wheat_2010_2024_master.csv
📊 Metrics for wheat_2010_2024_master.csv:
 MAE   : 316.08
 RMSE  : 671.66
 R²    : 0.5844
 MAPE  : 16.15%
 ✅ Accuracy: 83.85%


✅ ARIMAX Crop Forecasting Completed!
📁 Models, CSV, Graphs saved successfully.


In [ ]:
#SARIMAX

In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import gc
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings("ignore")

# =======================================================
# Paths
# =======================================================
input_folder = "dataset"
output_csv = "sarimax_crop_updated_csv_daily"
output_graphs = "sarimax_crop_graphs_daily"
output_metrics_csv = "sarimax_crop_metrics_daily.csv"

os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

metrics_list = []

# =======================================================
# Helper Functions
# =======================================================
def parse_dates_safe(date_series):
    """Safely parse various date formats"""
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    """MAPE calculation"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# =======================================================
# PROCESS EACH CROP FILE
# =======================================================
for file in os.listdir(input_folder):
    if not file.endswith(".csv"):
        continue

    print(f"\n========== Processing Crop File: {file} ==========")
    file_path = os.path.join(input_folder, file)

    # -----------------------------
    # Read and Prepare Data
    # -----------------------------
    df = pd.read_csv(file_path)
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')

    # ✅ Handle duplicate dates — aggregate using mean
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)

    df.set_index('Date', inplace=True)

    # Handle Missing Values
    df['Average Price'].fillna(method='ffill', inplace=True)
    df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)

    # ✅ Ensure Daily Frequency safely
    df = df.asfreq('D')
    df['Average Price'].fillna(method='ffill', inplace=True)

    # -----------------------------
    # ✅ Feature Engineering
    # -----------------------------
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_7'] = df['Average Price'].shift(7)
    df['MA_7'] = df['Average Price'].rolling(window=7).mean()
    df['MA_30'] = df['Average Price'].rolling(window=30).mean()

    df.fillna(method='bfill', inplace=True)

    # Exogenous variables (features)
    exog_data = df[['Lag_1', 'Lag_7', 'MA_7', 'MA_30']]

    # -----------------------------
    # ✅ Build SARIMAX Model
    # -----------------------------
    model = SARIMAX(
        df['Average Price'],
        exog=exog_data,
        order=(1, 1, 1),
        seasonal_order=(1, 1, 1, 7),
        enforce_stationarity=False,
        enforce_invertibility=False
    )

    model_fit = model.fit(disp=False)

    # -----------------------------
    # ✅ Predictions
    # -----------------------------
    df['Predicted'] = model_fit.predict(
        start=0,
        end=len(df) - 1,
        exog=exog_data
    )

    # ✅ Round numeric columns to 2 decimals
    df = df.round({'Average Price': 2, 'Predicted': 2, 'Lag_1': 2, 'Lag_7': 2, 'MA_7': 2, 'MA_30': 2})

    # -----------------------------
    # ✅ Metrics Calculation (Rounded)
    # -----------------------------
    y_true = df['Average Price']
    y_pred = df['Predicted']

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    accuracy = round(100 - mape, 2)

    print(f"MAE        : {mae}")
    print(f"RMSE       : {rmse}")
    print(f"R²         : {r2}")
    print(f"MAPE(%)    : {mape}")
    print(f"Accuracy(%) : {accuracy}\n")

    metrics_list.append({
        'File': file,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': accuracy
    })

    # -----------------------------
    # ✅ Save Updated CSV (Rounded)
    # -----------------------------
    updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_sarimax_daily_updated.csv"))
    df.to_csv(updated_csv_path, float_format='%.2f')

    # -----------------------------
    # ✅ Save Graph
    # -----------------------------
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df['Average Price'], label='Actual Price', color='blue')
    plt.plot(df.index, df['Predicted'], label='Predicted', color='red', linestyle='dashed')
    plt.plot(df.index, df['MA_7'], label='MA7', color='green', linestyle='--')
    plt.plot(df.index, df['MA_30'], label='MA30', color='orange', linestyle='--')
    plt.title(f"SARIMAX Daily Forecast - {file}")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True)

    graph_path = os.path.join(output_graphs, file.replace(".csv", "_sarimax_daily_graph.png"))
    plt.savefig(graph_path)
    plt.close()

    # ✅ Cleanup Memory
    del model, model_fit, df, exog_data
    gc.collect()

# =======================================================
# ✅ Save Final Metrics CSV (Rounded)
# =======================================================
metrics_df = pd.DataFrame(metrics_list)
metrics_df = metrics_df.round(2)
metrics_df.to_csv(output_metrics_csv, index=False, float_format='%.2f')

print("\n✅ All Crop Files Processed Successfully (Daily SARIMAX with Full Features + Rounded to 2 Decimals)!")
print(f"📊 Metrics saved to: {output_metrics_csv}")
print(f"📈 Graphs saved to: {output_graphs}")
print(f"📄 Updated CSVs saved to: {output_csv}")


========== Processing Crop File: capsicum_2010_2024_master.csv ==========
MAE        : 32.42
RMSE       : 120.03
R²         : 0.973
MAPE(%)    : 1.44
Accuracy(%) : 98.56


========== Processing Crop File: onion_2010_2024_master.csv ==========
MAE        : 12.36
RMSE       : 107.21
R²         : 0.9745
MAPE(%)    : 0.77
Accuracy(%) : 99.23


========== Processing Crop File: tomato_2010_2024_master.csv ==========
MAE        : 19.23
RMSE       : 37.8
R²         : 0.9976
MAPE(%)    : 1.42
Accuracy(%) : 98.58


========== Processing Crop File: wheat_2010_2024_master.csv ==========
MAE        : 11.06
RMSE       : 167.87
R²         : 0.8895
MAPE(%)    : 0.55
Accuracy(%) : 99.45


✅ All Crop Files Processed Successfully (Daily SARIMAX with Full Features + Rounded to 2 Decimals)!
📊 Metrics saved to: sarimax_crop_metrics_daily.csv
📈 Graphs saved to: sarimax_crop_graphs_daily
📄 Updated CSVs saved to: sarimax_crop_updated_csv_daily


In [ ]:
#TAT+MHA

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_output_models"
output_csv = "tat_output_csv"
output_graphs = "tat_output_graphs"
output_logs = "tat_output_logs"
metrics_file = "tat_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Config
# -----------------------------
look_back = 30
batch_size = 32
epochs = 60
val_split = 0.15
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# -----------------------------
# Helper functions
# -----------------------------
def parse_dates_safe(date_series):
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    """MAPE calculation"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def create_multivariate_dataset(y_scaled, exog_scaled, look_back):
    n = len(y_scaled)
    k = exog_scaled.shape[1] if exog_scaled is not None else 0
    X, Y = [], []
    for i in range(n - look_back):
        y_seq = y_scaled[i:i+look_back, 0]
        if k > 0:
            exog_seq = exog_scaled[i:i+look_back, :]
            combined = np.concatenate([y_seq.reshape(-1,1), exog_seq], axis=1)
        else:
            combined = y_seq.reshape(-1,1)
        X.append(combined)
        Y.append(y_scaled[i+look_back, 0])
    return np.array(X), np.array(Y)

# -----------------------------
# Positional encoding
# -----------------------------
def add_positional_encoding(x):
    seq_len = tf.shape(x)[1]
    d_model = tf.shape(x)[2]
    pos = tf.cast(tf.range(seq_len)[:, tf.newaxis], tf.float32)
    i = tf.cast(tf.range(d_model)[tf.newaxis, :], tf.float32)
    angle_rates = 1 / tf.pow(10000., (2 * (i//2)) / tf.cast(d_model, tf.float32))
    angle_rads = pos * angle_rates
    sines = tf.math.sin(angle_rads[:, 0::2])
    coses = tf.math.cos(angle_rads[:, 1::2])
    pos_encoding = tf.concat([sines, coses], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return x + tf.cast(pos_encoding, x.dtype)

# -----------------------------
# Transformer model (TAT + MHA)
# -----------------------------
def build_tat_mha_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    x = layers.Dense(d_model)(inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = layers.Lambda(add_positional_encoding)(x)

    attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
    attn_output = layers.Dropout(dropout_rate)(attn_output)
    out1 = layers.LayerNormalization(epsilon=1e-6)(x + attn_output)

    ffn = layers.Dense(ff_dim, activation='relu')(out1)
    ffn = layers.Dense(d_model)(ffn)
    ffn = layers.Dropout(dropout_rate)(ffn)
    out2 = layers.LayerNormalization(epsilon=1e-6)(out1 + ffn)

    attn_output2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(out2, out2)
    attn_output2 = layers.Dropout(dropout_rate)(attn_output2)
    out3 = layers.LayerNormalization(epsilon=1e-6)(out2 + attn_output2)

    x = layers.GlobalAveragePooling1D()(out3)
    x = layers.Dense(ff_dim//2, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1, name="pred")(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Process each crop file
# -----------------------------
metrics_list = []

for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue

    print(f"\n🚀 Processing: {file}")
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path)

    # Parse and sort dates
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')

    # Aggregate duplicate dates
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)

    # Handle missing prices
    df['Average Price'].fillna(method='ffill', inplace=True)
    df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)
    df = df.asfreq('D')
    df['Average Price'].fillna(method='ffill', inplace=True)

    # Feature engineering
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_7'] = df['Average Price'].shift(7)
    df['MA_7'] = df['Average Price'].rolling(7).mean()
    df['MA_30'] = df['Average Price'].rolling(30).mean()
    df.fillna(method='bfill', inplace=True)

    # Prepare scalers
    target_scaler = MinMaxScaler((0,1))
    exog_scaler = MinMaxScaler((0,1))

    y = df[['Average Price']].values.astype('float32')
    exog_cols = ['Lag_1','Lag_7','MA_7','MA_30']
    exog = df[exog_cols].values.astype('float32')

    y_scaled = target_scaler.fit_transform(y)
    exog_scaled = exog_scaler.fit_transform(exog)

    X_all, y_all = create_multivariate_dataset(y_scaled, exog_scaled, look_back)
    if len(X_all) == 0:
        print(f"⚠ Not enough data in {file}. Skipping.")
        continue

    val_size = int(len(X_all)*val_split)
    train_size = len(X_all) - val_size
    X_train, y_train = X_all[:train_size], y_all[:train_size]
    X_val, y_val = X_all[train_size:], y_all[train_size:]

    model = build_tat_mha_model((look_back, X_train.shape[2]), 64, 4, 128, 0.15)

    es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

    model.fit(X_train, y_train, validation_data=(X_val, y_val),
              epochs=epochs, batch_size=batch_size, callbacks=[es, rl], verbose=1)

    # Predict
    preds_scaled = model.predict(X_all, batch_size=batch_size)
    preds = target_scaler.inverse_transform(preds_scaled)
    preds = preds.flatten()

    df['Predicted'] = np.nan
    df.iloc[look_back:, df.columns.get_loc('Predicted')] = preds
    df = df.round(2)

    # Metrics
    mask = ~np.isnan(df['Predicted'])
    y_true = df.loc[mask, 'Average Price']
    y_pred = df.loc[mask, 'Predicted']

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': accuracy
    })

    # Save model + CSV + Graph
    model.save(os.path.join(output_models, file.replace(".csv", "_tat_final.h5")))
    df.reset_index().to_csv(os.path.join(output_csv, file.replace(".csv", "_tat_updated.csv")), index=False, float_format='%.2f')

    plt.figure(figsize=(12,6))
    plt.plot(df.index, df['Average Price'], label='Actual', color='blue')
    plt.plot(df.index, df['Predicted'], label='Predicted', color='red', linestyle='dashed')
    plt.plot(df.index, df['MA_7'], label='MA7', color='green', linestyle='--')
    plt.plot(df.index, df['MA_30'], label='MA30', color='orange', linestyle='--')
    plt.title(f"TAT+MHA Daily Forecast - {file}")
    plt.xlabel('Date')
    plt.ylabel('Average Price')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_graphs, file.replace(".csv", "_tat_graph.png")))
    plt.close()

    print(f"✅ Done {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={accuracy}%")

    del model, X_all, X_train, X_val, y_all, y_train, y_val
    gc.collect()

# -----------------------------
# ✅ Save all metrics in one CSV
# -----------------------------
metrics_df = pd.DataFrame(metrics_list)
metrics_df = metrics_df.round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 All Crops Processed! Metrics saved to {metrics_file}")


🚀 Processing: capsicum_2010_2024_master.csv

Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 11s 30ms/step - loss: 0.0526 - mae: 0.1412 - val_loss: 0.0486 - val_mae: 0.1947 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 0.0061 - mae: 0.0587 - val_loss: 0.0048 - val_mae: 0.0463 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.0039 - mae: 0.0478 - val_loss: 0.0087 - val_mae: 0.0799 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.0038 - mae: 0.0463 - val_loss: 0.0032 - val_mae: 0.0380 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 0.0032 - mae: 0.0417 - val_loss: 0.0023 - val_mae: 0.0327 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 0.0029 - mae: 0.0404 - val_loss: 0.0034 - val_mae: 0.0454 - learning_rate: 0.0010
Epoch 7/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 0.0028 - mae: 0.0396 - val_loss: 0

✅ Done capsicum_2010_2024_master.csv | MAE=59.88, RMSE=84.53, R²=0.9864, MAPE=2.44%, Accuracy=97.56%

🚀 Processing: onion_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 23s 47ms/step - loss: 0.0797 - mae: 0.1570 - val_loss: 0.0304 - val_mae: 0.1211 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - loss: 0.0058 - mae: 0.0511 - val_loss: 0.0140 - val_mae: 0.0853 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 0.0034 - mae: 0.0397 - val_loss: 0.0087 - val_mae: 0.0736 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 0.0025 - mae: 0.0350 - val_loss: 0.0034 - val_mae: 0.0458 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - loss: 0.0021 - mae: 0.0325 - val_loss: 0.0030 - val_mae: 0.0377 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - loss: 0.0021 - mae: 0.0324 - val_loss: 0.0130 - val_mae: 0.0961 - learning_rate: 0.001

✅ Done onion_2010_2024_master.csv | MAE=26.28, RMSE=45.83, R²=0.9954, MAPE=1.45%, Accuracy=98.55%

🚀 Processing: tomato_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - loss: 0.0544 - mae: 0.1112 - val_loss: 0.0229 - val_mae: 0.0960 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 0.0022 - mae: 0.0312 - val_loss: 0.0157 - val_mae: 0.0818 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.0014 - mae: 0.0244 - val_loss: 0.0223 - val_mae: 0.1185 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.0012 - mae: 0.0233 - val_loss: 0.0087 - val_mae: 0.0615 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 9.5324e-04 - mae: 0.0208 - val_loss: 0.0057 - val_mae: 0.0478 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 8.6030e-04 - mae: 0.0198 - val_loss: 0.0083 - val_mae: 0.0661 - learning_rate:

✅ Done tomato_2010_2024_master.csv | MAE=60.39, RMSE=127.39, R²=0.973, MAPE=4.14%, Accuracy=95.86%

🚀 Processing: wheat_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 22s 56ms/step - loss: 0.0516 - mae: 0.1365 - val_loss: 0.0830 - val_mae: 0.2756 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - loss: 0.0042 - mae: 0.0467 - val_loss: 0.0326 - val_mae: 0.1652 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - loss: 0.0032 - mae: 0.0396 - val_loss: 0.0169 - val_mae: 0.1112 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - loss: 0.0033 - mae: 0.0405 - val_loss: 0.0315 - val_mae: 0.1631 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.0029 - mae: 0.0384 - val_loss: 0.0234 - val_mae: 0.1382 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 0.0026 - mae: 0.0356 - val_loss: 0.0132 - val_mae: 0.0957 - learning_rate: 0.0010


✅ Done wheat_2010_2024_master.csv | MAE=58.82, RMSE=99.73, R²=0.9609, MAPE=2.32%, Accuracy=97.68%

📊 All Crops Processed! Metrics saved to tat_metrics.csv


In [ ]:
#TAT+MQA

In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# ======================================================
# Output folders
# ======================================================
input_folder = "dataset"
output_models = "tat_mqa_output_models"
output_csv = "tat_mqa_output_csv"
output_graphs = "tat_mqa_output_graphs"
output_logs = "tat_mqa_output_logs"
metrics_file = "tat_mqa_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# ======================================================
# Config
# ======================================================
look_back = 30
batch_size = 32
epochs = 60
val_split = 0.15
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# ======================================================
# Helpers
# ======================================================
def parse_dates_safe(date_series):
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def create_multivariate_dataset(y_scaled, exog_scaled, look_back):
    n = len(y_scaled)
    k = exog_scaled.shape[1]
    X, Y = [], []
    for i in range(n - look_back):
        y_seq = y_scaled[i:i+look_back, 0]
        exog_seq = exog_scaled[i:i+look_back, :]
        combined = np.concatenate([y_seq.reshape(-1,1), exog_seq], axis=1)
        X.append(combined)
        Y.append(y_scaled[i+look_back, 0])
    return np.array(X), np.array(Y)

# ======================================================
# Multi-Query Attention Layer
# ======================================================
class MultiQueryAttention(layers.Layer):
    def __init__(self, num_heads=4, key_dim=64, dropout_rate=0.1, **kwargs):
        super(MultiQueryAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.q_dense = [layers.Dense(key_dim) for _ in range(num_heads)]
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.dropout = layers.Dropout(dropout_rate)
        self.output_dense = layers.Dense(key_dim)

    def call(self, x):
        K = self.k_dense(x)
        V = self.v_dense(x)
        head_outputs = []
        for q_layer in self.q_dense:
            Q = q_layer(x)
            scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
            attention_weights = tf.nn.softmax(scores, axis=-1)
            attention_output = tf.matmul(attention_weights, V)
            head_outputs.append(attention_output)
        concat = tf.concat(head_outputs, axis=-1)
        output = self.output_dense(concat)
        output = self.dropout(output)
        return output

# ======================================================
# Build TAT + MQA Model
# ======================================================
def build_tat_mqa_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)              # (seq, features)
    # Project inputs to d_model for dimension alignment
    inputs_proj = layers.Dense(d_model)(inputs)           # (seq, d_model)
    
    attn_out = MultiQueryAttention(num_heads=num_heads, key_dim=d_model, dropout_rate=dropout_rate)(inputs_proj)
    attn_out = layers.LayerNormalization(epsilon=1e-6)(attn_out + inputs_proj)

    # Feed-forward block
    ff = layers.Dense(ff_dim, activation='relu')(attn_out)
    ff = layers.Dense(d_model)(ff)
    ff = layers.Dropout(dropout_rate)(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(ff + attn_out)

    # Pool and output
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(ff_dim//2, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# ======================================================
# Main Loop
# ======================================================
metrics_list = []

for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue

    print(f"\n🚀 Processing: {file}")
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path)

    # Date handling
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)

    # Fill missing
    df['Average Price'] = df['Average Price'].ffill().bfill().fillna(df['Average Price'].mean())
    df = df.asfreq('D')
    df['Average Price'] = df['Average Price'].ffill().bfill()

    # Feature engineering
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_7'] = df['Average Price'].shift(7)
    df['MA_7'] = df['Average Price'].rolling(7).mean()
    df['MA_30'] = df['Average Price'].rolling(30).mean()
    df = df.bfill()

    # Scaling
    target_scaler = MinMaxScaler((0,1))
    exog_scaler = MinMaxScaler((0,1))
    y = df[['Average Price']].values.astype('float32')
    exog = df[['Lag_1','Lag_7','MA_7','MA_30']].values.astype('float32')
    y_scaled = target_scaler.fit_transform(y)
    exog_scaled = exog_scaler.fit_transform(exog)

    X_all, y_all = create_multivariate_dataset(y_scaled, exog_scaled, look_back)
    if len(X_all) == 0:
        print(f"⚠ Not enough data for {file}")
        continue

    val_size = int(len(X_all) * val_split)
    X_train, y_train = X_all[:-val_size], y_all[:-val_size]
    X_val, y_val = X_all[-val_size:], y_all[-val_size:]

    # Build & Train
    model = build_tat_mqa_model((look_back, X_train.shape[2]), d_model=64)
    es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

    model.fit(X_train, y_train, validation_data=(X_val, y_val),
              epochs=epochs, batch_size=batch_size, callbacks=[es, rl], verbose=1)

    # Predict
    preds_scaled = model.predict(X_all, batch_size=batch_size)
    preds = target_scaler.inverse_transform(preds_scaled).flatten()
    df['Predicted'] = np.nan
    df.iloc[look_back:, df.columns.get_loc('Predicted')] = preds
    df = df.round(2)

    # Metrics
    mask = ~np.isnan(df['Predicted'])
    y_true = df.loc[mask, 'Average Price']
    y_pred = df.loc[mask, 'Predicted']
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': accuracy
    })

    # Save model & CSV
    model.save(os.path.join(output_models, file.replace(".csv", "_tat_mqa_model.h5")))
    df.reset_index().to_csv(os.path.join(output_csv, file.replace(".csv", "_tat_mqa_updated.csv")), index=False, float_format='%.2f')

    # Graph
    plt.figure(figsize=(12,6))
    plt.plot(df.index, df['Average Price'], label='Actual', color='blue')
    plt.plot(df.index, df['Predicted'], label='Predicted', color='red', linestyle='dashed')
    plt.plot(df.index, df['MA_7'], label='MA7', color='green', linestyle='--')
    plt.plot(df.index, df['MA_30'], label='MA30', color='orange', linestyle='--')
    plt.title(f"TAT+MQA Daily Forecast - {file}")
    plt.xlabel('Date')
    plt.ylabel('Average Price')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_graphs, file.replace(".csv", "_tat_mqa_graph.png")), bbox_inches='tight')
    plt.close()

    print(f"✅ {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={accuracy}%")
    del model, X_all, X_train, X_val, y_all, y_train, y_val
    gc.collect()

# ======================================================
# Save Metrics
# ======================================================
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 All Crops Processed! Metrics saved to {metrics_file}")
print(metrics_df)


🚀 Processing: capsicum_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 0.0283 - mae: 0.1123 - val_loss: 0.0143 - val_mae: 0.0949 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0072 - mae: 0.0648 - val_loss: 0.0130 - val_mae: 0.0929 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0062 - mae: 0.0597 - val_loss: 0.0075 - val_mae: 0.0682 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0054 - mae: 0.0548 - val_loss: 0.0109 - val_mae: 0.0849 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0044 - mae: 0.0496 - val_loss: 0.0061 - val_mae: 0.0611 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0043 - mae: 0.0486 - val_loss: 0.0054 - val_mae: 0.0560 - learning_rate: 0.0010
Epoch 7/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0039 - mae: 0.0464 - val_loss: 0.

✅ capsicum_2010_2024_master.csv | MAE=88.06, RMSE=122.97, R²=0.9711, MAPE=3.53%, Accuracy=96.47%

🚀 Processing: onion_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - loss: 0.0582 - mae: 0.1328 - val_loss: 0.0130 - val_mae: 0.0954 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0047 - mae: 0.0470 - val_loss: 0.0045 - val_mae: 0.0539 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0037 - mae: 0.0423 - val_loss: 0.0060 - val_mae: 0.0580 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0028 - mae: 0.0370 - val_loss: 0.0022 - val_mae: 0.0348 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0026 - mae: 0.0356 - val_loss: 0.0046 - val_mae: 0.0460 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0024 - mae: 0.0335 - val_loss: 0.0018 - val_mae: 0.0311 - learning_rate: 0.0010
Ep

✅ onion_2010_2024_master.csv | MAE=54.92, RMSE=80.34, R²=0.9857, MAPE=3.34%, Accuracy=96.66%

🚀 Processing: tomato_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - loss: 0.0531 - mae: 0.1143 - val_loss: 0.0150 - val_mae: 0.0709 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0024 - mae: 0.0321 - val_loss: 0.0085 - val_mae: 0.0563 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0018 - mae: 0.0282 - val_loss: 0.0049 - val_mae: 0.0457 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0013 - mae: 0.0253 - val_loss: 0.0059 - val_mae: 0.0587 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0013 - mae: 0.0247 - val_loss: 0.0067 - val_mae: 0.0639 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0012 - mae: 0.0230 - val_loss: 0.0051 - val_mae: 0.0507 - learning_rate: 0.0010
Epoch

✅ tomato_2010_2024_master.csv | MAE=117.42, RMSE=205.87, R²=0.9294, MAPE=8.53%, Accuracy=91.47%

🚀 Processing: wheat_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - loss: 0.1037 - mae: 0.1774 - val_loss: 0.0347 - val_mae: 0.1674 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0054 - mae: 0.0555 - val_loss: 0.0245 - val_mae: 0.1397 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0041 - mae: 0.0472 - val_loss: 0.0196 - val_mae: 0.1235 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0039 - mae: 0.0451 - val_loss: 0.0134 - val_mae: 0.0974 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0036 - mae: 0.0429 - val_loss: 0.0130 - val_mae: 0.0972 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0033 - mae: 0.0407 - val_loss: 0.0162 - val_mae: 0.1127 - learning_rate: 0.0010
Epo

✅ wheat_2010_2024_master.csv | MAE=75.46, RMSE=105.76, R²=0.9561, MAPE=3.15%, Accuracy=96.85%

📊 All Crops Processed! Metrics saved to tat_mqa_metrics.csv
                        Crop     MAE    RMSE    R2  MAPE(%)  Accuracy(%)
0  capsicum_2010_2024_master   88.06  122.97  0.97     3.53        96.47
1     onion_2010_2024_master   54.92   80.34  0.99     3.34        96.66
2    tomato_2010_2024_master  117.42  205.87  0.93     8.53        91.47
3     wheat_2010_2024_master   75.46  105.76  0.96     3.15        96.85


In [ ]:
#TAT+GQA

In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# ======================================================
# Output directories
# ======================================================
input_folder = "dataset"
output_models = "tat_gqa_output_models"
output_csv = "tat_gqa_output_csv"
output_graphs = "tat_gqa_output_graphs"
output_logs = "tat_gqa_output_logs"
metrics_file = "tat_gqa_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# ======================================================
# Config
# ======================================================
look_back = 30
batch_size = 32
epochs = 60
val_split = 0.15
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# ======================================================
# Helper Functions
# ======================================================
def parse_dates_safe(date_series):
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def create_multivariate_dataset(y_scaled, exog_scaled, look_back):
    n = len(y_scaled)
    k = exog_scaled.shape[1]
    X, Y = [], []
    for i in range(n - look_back):
        y_seq = y_scaled[i:i+look_back, 0]
        exog_seq = exog_scaled[i:i+look_back, :]
        combined = np.concatenate([y_seq.reshape(-1,1), exog_seq], axis=1)
        X.append(combined)
        Y.append(y_scaled[i+look_back, 0])
    return np.array(X), np.array(Y)

# ======================================================
# Grouped Query Attention Layer
# ======================================================
class GroupedQueryAttention(layers.Layer):
    def __init__(self, num_groups=2, key_dim=64, dropout_rate=0.1, **kwargs):
        super(GroupedQueryAttention, self).__init__(**kwargs)
        self.num_groups = num_groups
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.q_dense_groups = [layers.Dense(key_dim) for _ in range(num_groups)]
        self.dropout = layers.Dropout(dropout_rate)
        self.output_dense = layers.Dense(key_dim)

    def call(self, x):
        K = self.k_dense(x)
        V = self.v_dense(x)
        group_outputs = []
        for q_layer in self.q_dense_groups:
            Q = q_layer(x)
            scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
            attention_weights = tf.nn.softmax(scores, axis=-1)
            attn_output = tf.matmul(attention_weights, V)
            group_outputs.append(attn_output)
        concat = tf.concat(group_outputs, axis=-1)
        output = self.output_dense(concat)
        return self.dropout(output)

# ======================================================
# Build TAT + GQA model
# ======================================================
def build_tat_gqa_model(input_shape, d_model=64, num_groups=2, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    # Project inputs to d_model for residual add compatibility
    inputs_proj = layers.Dense(d_model)(inputs)
    x = GroupedQueryAttention(num_groups=num_groups, key_dim=d_model, dropout_rate=dropout_rate)(inputs_proj)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs_proj)

    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(d_model)(ff)
    ff = layers.Dropout(dropout_rate)(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(ff_dim//2, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

# ======================================================
# Main Loop
# ======================================================
metrics_list = []

for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue

    print(f"\n🚀 Processing: {file}")
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path)

    # Prepare data
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)

    # Fill missing
    df['Average Price'] = df['Average Price'].ffill().bfill().fillna(df['Average Price'].mean())
    df = df.asfreq('D')
    df['Average Price'] = df['Average Price'].ffill().bfill()

    # Feature Engineering
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_7'] = df['Average Price'].shift(7)
    df['MA_7'] = df['Average Price'].rolling(7).mean()
    df['MA_30'] = df['Average Price'].rolling(30).mean()
    df = df.bfill()

    # Scaling
    target_scaler = MinMaxScaler((0,1))
    exog_scaler = MinMaxScaler((0,1))
    y = df[['Average Price']].values.astype('float32')
    exog = df[['Lag_1','Lag_7','MA_7','MA_30']].values.astype('float32')
    y_scaled = target_scaler.fit_transform(y)
    exog_scaled = exog_scaler.fit_transform(exog)

    X_all, y_all = create_multivariate_dataset(y_scaled, exog_scaled, look_back)
    if len(X_all) == 0:
        print(f"⚠ Not enough data for {file}")
        continue

    val_size = int(len(X_all)*val_split)
    X_train, y_train = X_all[:-val_size], y_all[:-val_size]
    X_val, y_val = X_all[-val_size:], y_all[-val_size:]

    # Train model
    model = build_tat_gqa_model((look_back, X_train.shape[2]), d_model=64, num_groups=2)
    es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)
    model.fit(X_train, y_train, validation_data=(X_val, y_val),
              epochs=epochs, batch_size=batch_size, callbacks=[es, rl], verbose=1)

    # Predict
    preds_scaled = model.predict(X_all, batch_size=batch_size)
    preds = target_scaler.inverse_transform(preds_scaled).flatten()
    df['Predicted'] = np.nan
    df.iloc[look_back:, df.columns.get_loc('Predicted')] = preds
    df = df.round(2)

    # Metrics
    mask = ~np.isnan(df['Predicted'])
    y_true = df.loc[mask, 'Average Price']
    y_pred = df.loc[mask, 'Predicted']
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': accuracy
    })

    # Save model & CSV
    model.save(os.path.join(output_models, file.replace(".csv", "_tat_gqa_model.h5")))
    df.reset_index().to_csv(os.path.join(output_csv, file.replace(".csv", "_tat_gqa_updated.csv")), index=False, float_format='%.2f')

    # Graph
    plt.figure(figsize=(12,6))
    plt.plot(df.index, df['Average Price'], label='Actual', color='blue')
    plt.plot(df.index, df['Predicted'], label='Predicted', color='red', linestyle='dashed')
    plt.plot(df.index, df['MA_7'], label='MA7', color='green', linestyle='--')
    plt.plot(df.index, df['MA_30'], label='MA30', color='orange', linestyle='--')
    plt.title(f"TAT+GQA Daily Forecast - {file}")
    plt.xlabel('Date')
    plt.ylabel('Average Price')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_graphs, file.replace(".csv", "_tat_gqa_graph.png")), bbox_inches='tight')
    plt.close()

    print(f"✅ {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={accuracy}%")
    del model, X_all, X_train, X_val, y_all, y_train, y_val
    gc.collect()

# ======================================================
# Save Metrics
# ======================================================
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 All Crops Processed! Metrics saved to {metrics_file}")
print(metrics_df)


🚀 Processing: capsicum_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - loss: 0.0721 - mae: 0.1464 - val_loss: 0.0114 - val_mae: 0.0791 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0080 - mae: 0.0683 - val_loss: 0.0068 - val_mae: 0.0622 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0070 - mae: 0.0630 - val_loss: 0.0068 - val_mae: 0.0637 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0063 - mae: 0.0602 - val_loss: 0.0062 - val_mae: 0.0592 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0059 - mae: 0.0578 - val_loss: 0.0060 - val_mae: 0.0585 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0056 - mae: 0.0563 - val_loss: 0.0093 - val_mae: 0.0772 - learning_rate: 0.0010
Epoch 7/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0052 - mae: 0.0535 - val_loss: 0.

✅ capsicum_2010_2024_master.csv | MAE=102.9, RMSE=141.55, R²=0.9618, MAPE=4.04%, Accuracy=95.96%

🚀 Processing: onion_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - loss: 0.0306 - mae: 0.0988 - val_loss: 0.0072 - val_mae: 0.0625 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0036 - mae: 0.0415 - val_loss: 0.0032 - val_mae: 0.0406 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0026 - mae: 0.0355 - val_loss: 0.0016 - val_mae: 0.0317 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0029 - mae: 0.0362 - val_loss: 0.0014 - val_mae: 0.0287 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0023 - mae: 0.0326 - val_loss: 0.0019 - val_mae: 0.0331 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0023 - mae: 0.0325 - val_loss: 0.0019 - val_mae: 0.0319 - learning_rate: 0.0010
Epo

✅ onion_2010_2024_master.csv | MAE=81.79, RMSE=114.92, R²=0.9708, MAPE=5.0%, Accuracy=95.0%

🚀 Processing: tomato_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - loss: 0.0292 - mae: 0.0891 - val_loss: 0.0108 - val_mae: 0.0547 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0021 - mae: 0.0318 - val_loss: 0.0069 - val_mae: 0.0460 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0017 - mae: 0.0290 - val_loss: 0.0052 - val_mae: 0.0469 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0014 - mae: 0.0260 - val_loss: 0.0044 - val_mae: 0.0401 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0012 - mae: 0.0240 - val_loss: 0.0041 - val_mae: 0.0417 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0012 - mae: 0.0229 - val_loss: 0.0042 - val_mae: 0.0440 - learning_rate: 0.0010
Epoch 7

✅ tomato_2010_2024_master.csv | MAE=136.47, RMSE=228.63, R²=0.913, MAPE=10.54%, Accuracy=89.46%

🚀 Processing: wheat_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - loss: 0.0453 - mae: 0.1264 - val_loss: 0.0255 - val_mae: 0.1396 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0046 - mae: 0.0482 - val_loss: 0.0102 - val_mae: 0.0805 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0037 - mae: 0.0434 - val_loss: 0.0127 - val_mae: 0.0918 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0036 - mae: 0.0427 - val_loss: 0.0163 - val_mae: 0.1134 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0033 - mae: 0.0407 - val_loss: 0.0057 - val_mae: 0.0565 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0031 - mae: 0.0394 - val_loss: 0.0125 - val_mae: 0.0931 - learning_rate: 0.0010
Epo

✅ wheat_2010_2024_master.csv | MAE=108.36, RMSE=135.05, R²=0.9284, MAPE=4.71%, Accuracy=95.29%

📊 All Crops Processed! Metrics saved to tat_gqa_metrics.csv
                        Crop     MAE    RMSE    R2  MAPE(%)  Accuracy(%)
0  capsicum_2010_2024_master  102.90  141.55  0.96     4.04        95.96
1     onion_2010_2024_master   81.79  114.92  0.97     5.00        95.00
2    tomato_2010_2024_master  136.47  228.63  0.91    10.54        89.46
3     wheat_2010_2024_master  108.36  135.05  0.93     4.71        95.29


In [ ]:
#TAT+HA

In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# ======================================================
# Output directories
# ======================================================
input_folder = "dataset"
output_models = "tat_ha_output_models"
output_csv = "tat_ha_output_csv"
output_graphs = "tat_ha_output_graphs"
metrics_file = "tat_ha_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# ======================================================
# Config
# ======================================================
look_back = 30
batch_size = 32
epochs = 60
val_split = 0.15
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# ======================================================
# Helper Functions
# ======================================================
def parse_dates_safe(date_series):
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def create_multivariate_dataset(y_scaled, exog_scaled, look_back):
    n = len(y_scaled)
    X, Y = [], []
    for i in range(n - look_back):
        seq_y = y_scaled[i:i+look_back, 0]
        seq_exog = exog_scaled[i:i+look_back, :]
        seq = np.concatenate([seq_y.reshape(-1,1), seq_exog], axis=1)
        X.append(seq)
        Y.append(y_scaled[i+look_back, 0])
    return np.array(X), np.array(Y)

# ======================================================
# Fixed Hierarchical Attention Layer
# ======================================================
@tf.keras.utils.register_keras_serializable()
class HierarchicalAttention(layers.Layer):
    def __init__(self, d_model=64, num_heads=4, dropout_rate=0.15, ff_dim=128, **kwargs):
        super().__init__(**kwargs)
        self.d_model = d_model
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate
        self.ff_dim = ff_dim

        self.feature_proj = layers.Dense(d_model)
        self.feature_score = layers.Dense(1)
        self.temporal_mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ff1 = layers.Dense(ff_dim, activation='relu')
        self.ff2 = layers.Dense(d_model)
        self.ln1 = layers.LayerNormalization(epsilon=1e-6)
        self.ln2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout = layers.Dropout(dropout_rate)

    def build(self, input_shape):
        super().build(input_shape)

    @tf.function
    def call(self, x, training=False):
        # Feature-level attention — score each feature
        scores = self.feature_score(x)  # (batch, seq, features, 1)? No — Keras auto flattens => (batch, seq, 1)
        attn_weights = tf.nn.softmax(scores, axis=1)  # across time dimension

        # Weighted aggregation
        weighted = x * attn_weights
        feature_agg = tf.reduce_sum(weighted, axis=-1, keepdims=True)
        seq_repr = self.feature_proj(feature_agg)

        # Temporal attention
        attn_out = self.temporal_mha(seq_repr, seq_repr)
        attn_out = self.dropout(attn_out, training=training)
        out1 = self.ln1(seq_repr + attn_out)

        # Feed-forward network
        ffn = self.ff1(out1)
        ffn = self.ff2(ffn)
        ffn = self.dropout(ffn, training=training)
        out2 = self.ln2(out1 + ffn)
        return out2

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], self.d_model)

# ======================================================
# Model Builder
# ======================================================
def build_tat_ha_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.15):
    inputs = layers.Input(shape=input_shape)
    inputs_proj = layers.Dense(d_model)(inputs)
    ha_out = HierarchicalAttention(d_model=d_model, num_heads=num_heads, ff_dim=ff_dim, dropout_rate=dropout_rate)(inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(ha_out + inputs_proj)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(ff_dim//2, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1)(x)
    model = models.Model(inputs, outputs)
    model.compile(optimizer=optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

# ======================================================
# Main Loop
# ======================================================
metrics_list = []

for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue
    print(f"\n🚀 Processing: {file}")
    path = os.path.join(input_folder, file)
    df = pd.read_csv(path)

    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)
    df['Average Price'] = df['Average Price'].ffill().bfill().fillna(df['Average Price'].mean())
    df = df.asfreq('D')
    df['Average Price'] = df['Average Price'].ffill().bfill()

    # Feature Engineering
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_7'] = df['Average Price'].shift(7)
    df['MA_7'] = df['Average Price'].rolling(7).mean()
    df['MA_30'] = df['Average Price'].rolling(30).mean()
    df = df.bfill()

    # Prepare data
    y = df[['Average Price']].values.astype('float32')
    exog = df[['Lag_1','Lag_7','MA_7','MA_30']].values.astype('float32')
    y_scaler, exog_scaler = MinMaxScaler(), MinMaxScaler()
    y_scaled = y_scaler.fit_transform(y)
    exog_scaled = exog_scaler.fit_transform(exog)

    X_all, y_all = create_multivariate_dataset(y_scaled, exog_scaled, look_back)
    if len(X_all) == 0:
        continue

    val_size = int(len(X_all)*val_split)
    X_train, y_train = X_all[:-val_size], y_all[:-val_size]
    X_val, y_val = X_all[-val_size:], y_all[-val_size:]

    model = build_tat_ha_model((look_back, X_train.shape[2]))
    es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

    model.fit(X_train, y_train, validation_data=(X_val, y_val),
              epochs=epochs, batch_size=batch_size, callbacks=[es, rl], verbose=1)

    preds_scaled = model.predict(X_all)
    preds = y_scaler.inverse_transform(preds_scaled).flatten()
    df['Predicted'] = np.nan
    df.iloc[look_back:, df.columns.get_loc('Predicted')] = preds
    df = df.round(2)

    mask = ~np.isnan(df['Predicted'])
    y_true, y_pred = df.loc[mask, 'Average Price'], df.loc[mask, 'Predicted']
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    acc = round(100 - mape, 2)
    metrics_list.append({'Crop': file.replace('.csv',''), 'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE(%)': mape, 'Accuracy(%)': acc})

    model.save(os.path.join(output_models, file.replace('.csv','_tat_ha_model.h5')))
    df.reset_index().to_csv(os.path.join(output_csv, file.replace('.csv','_tat_ha_updated.csv')), index=False)

    plt.figure(figsize=(12,6))
    plt.plot(df.index, df['Average Price'], label='Actual', color='blue')
    plt.plot(df.index, df['Predicted'], label='Predicted', color='red', linestyle='dashed')
    plt.plot(df.index, df['MA_7'], label='MA7', color='green', linestyle='--')
    plt.plot(df.index, df['MA_30'], label='MA30', color='orange', linestyle='--')
    plt.title(f"TAT+HA Daily Forecast - {file}")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_graphs, file.replace('.csv','_tat_ha_graph.png')))
    plt.close()

    print(f"✅ {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={acc}%")
    gc.collect()

metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 Metrics saved to {metrics_file}")
print(metrics_df)


🚀 Processing: capsicum_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 0.0329 - mae: 0.1035 - val_loss: 0.0118 - val_mae: 0.0838 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0065 - mae: 0.0615 - val_loss: 0.0103 - val_mae: 0.0774 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0067 - mae: 0.0620 - val_loss: 0.0096 - val_mae: 0.0729 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0062 - mae: 0.0594 - val_loss: 0.0099 - val_mae: 0.0754 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0058 - mae: 0.0580 - val_loss: 0.0098 - val_mae: 0.0754 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0055 - mae: 0.0570 - val_loss: 0.0107 - val_mae: 0.0799 - learning_rate: 0.0010
Epoch 7/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0054 - mae: 0.0560 - val_loss: 0.

✅ capsicum_2010_2024_master.csv | MAE=208.07, RMSE=280.47, R²=0.8499, MAPE=8.53%, Accuracy=91.47%

🚀 Processing: onion_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 19s 26ms/step - loss: 0.0989 - mae: 0.1619 - val_loss: 0.0067 - val_mae: 0.0604 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0056 - mae: 0.0522 - val_loss: 0.0055 - val_mae: 0.0538 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0049 - mae: 0.0475 - val_loss: 0.0048 - val_mae: 0.0494 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0049 - mae: 0.0470 - val_loss: 0.0047 - val_mae: 0.0501 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.0044 - mae: 0.0448 - val_loss: 0.0047 - val_mae: 0.0500 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0043 - mae: 0.0437 - val_loss: 0.0042 - val_mae: 0.0454 - learning_rate: 0.0010
E

✅ onion_2010_2024_master.csv | MAE=153.58, RMSE=212.57, R²=0.9001, MAPE=9.68%, Accuracy=90.32%

🚀 Processing: tomato_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - loss: 0.0536 - mae: 0.1238 - val_loss: 0.0126 - val_mae: 0.0735 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0028 - mae: 0.0380 - val_loss: 0.0123 - val_mae: 0.0736 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0025 - mae: 0.0357 - val_loss: 0.0119 - val_mae: 0.0712 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0024 - mae: 0.0346 - val_loss: 0.0112 - val_mae: 0.0688 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0022 - mae: 0.0329 - val_loss: 0.0109 - val_mae: 0.0701 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0021 - mae: 0.0316 - val_loss: 0.0105 - val_mae: 0.0662 - learning_rate: 0.0010
Epo

✅ tomato_2010_2024_master.csv | MAE=166.6, RMSE=227.71, R²=0.9137, MAPE=14.86%, Accuracy=85.14%

🚀 Processing: wheat_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - loss: 0.0844 - mae: 0.1523 - val_loss: 0.0016 - val_mae: 0.0240 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0034 - mae: 0.0438 - val_loss: 0.0013 - val_mae: 0.0227 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0027 - mae: 0.0384 - val_loss: 0.0015 - val_mae: 0.0257 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0026 - mae: 0.0373 - val_loss: 0.0015 - val_mae: 0.0232 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0026 - mae: 0.0365 - val_loss: 0.0014 - val_mae: 0.0248 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0022 - mae: 0.0344 - val_loss: 0.0033 - val_mae: 0.0436 - learning_rate: 0.0010
Epo

✅ wheat_2010_2024_master.csv | MAE=54.72, RMSE=67.61, R²=0.982, MAPE=2.73%, Accuracy=97.27%

📊 Metrics saved to tat_ha_metrics.csv
                        Crop     MAE    RMSE    R2  MAPE(%)  Accuracy(%)
0  capsicum_2010_2024_master  208.07  280.47  0.85     8.53        91.47
1     onion_2010_2024_master  153.58  212.57  0.90     9.68        90.32
2    tomato_2010_2024_master  166.60  227.71  0.91    14.86        85.14
3     wheat_2010_2024_master   54.72   67.61  0.98     2.73        97.27


In [ ]:
#LSTM

In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# ======================================================
# Output directories
# ======================================================
input_folder = "dataset"
output_models = "lstm_output_models"
output_csv = "lstm_output_csv"
output_graphs = "lstm_output_graphs"
metrics_file = "lstm_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# ======================================================
# Config
# ======================================================
look_back = 30
batch_size = 32
epochs = 60
val_split = 0.15
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# ======================================================
# Helper Functions
# ======================================================
def parse_dates_safe(date_series):
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def create_multivariate_dataset(y_scaled, exog_scaled, look_back):
    n = len(y_scaled)
    X, Y = [], []
    for i in range(n - look_back):
        seq_y = y_scaled[i:i+look_back, 0]
        seq_exog = exog_scaled[i:i+look_back, :]
        seq = np.concatenate([seq_y.reshape(-1,1), seq_exog], axis=1)
        X.append(seq)
        Y.append(y_scaled[i+look_back, 0])
    return np.array(X), np.array(Y)

# ======================================================
# Build LSTM Model
# ======================================================
def build_lstm_model(input_shape, units=64, dropout_rate=0.2, lr=0.001):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.LSTM(units, return_sequences=True),
        layers.Dropout(dropout_rate),
        layers.LSTM(units//2),
        layers.Dropout(dropout_rate),
        layers.Dense(units//2, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer=optimizers.Adam(learning_rate=lr), loss='mse', metrics=['mae'])
    return model

# ======================================================
# Main loop
# ======================================================
metrics_list = []

for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue
    print(f"\n🚀 Processing: {file}")
    path = os.path.join(input_folder, file)
    df = pd.read_csv(path)

    # --- Handle Dates ---
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)

    # --- Fill and align data ---
    df['Average Price'] = df['Average Price'].ffill().bfill().fillna(df['Average Price'].mean())
    df = df.asfreq('D')
    df['Average Price'] = df['Average Price'].ffill().bfill()

    # --- Feature Engineering ---
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_7'] = df['Average Price'].shift(7)
    df['MA_7'] = df['Average Price'].rolling(window=7).mean()
    df['MA_30'] = df['Average Price'].rolling(window=30).mean()
    df = df.bfill()

    # --- Scaling ---
    y = df[['Average Price']].values.astype('float32')
    exog = df[['Lag_1', 'Lag_7', 'MA_7', 'MA_30']].values.astype('float32')

    y_scaler, exog_scaler = MinMaxScaler(), MinMaxScaler()
    y_scaled = y_scaler.fit_transform(y)
    exog_scaled = exog_scaler.fit_transform(exog)

    # --- Sequence Creation ---
    X_all, y_all = create_multivariate_dataset(y_scaled, exog_scaled, look_back)
    if len(X_all) == 0:
        continue

    # --- Train / Validation Split ---
    val_size = int(len(X_all) * val_split)
    X_train, y_train = X_all[:-val_size], y_all[:-val_size]
    X_val, y_val = X_all[-val_size:], y_all[-val_size:]

    # --- Build and Train ---
    model = build_lstm_model((look_back, X_train.shape[2]))
    es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

    model.fit(X_train, y_train, validation_data=(X_val, y_val),
              epochs=epochs, batch_size=batch_size, callbacks=[es, rl], verbose=1)

    # --- Predictions ---
    preds_scaled = model.predict(X_all)
    preds = y_scaler.inverse_transform(preds_scaled).flatten()

    df['Predicted'] = np.nan
    df.iloc[look_back:, df.columns.get_loc('Predicted')] = preds
    df = df.round(2)

    # --- Metrics ---
    mask = ~np.isnan(df['Predicted'])
    y_true, y_pred = df.loc[mask, 'Average Price'], df.loc[mask, 'Predicted']

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    acc = round(100 - mape, 2)

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': acc
    })

    # --- Save Outputs ---
    model.save(os.path.join(output_models, file.replace('.csv','_lstm_model.h5')))
    df_reset = df.reset_index()
    df_reset.to_csv(os.path.join(output_csv, file.replace('.csv','_lstm_updated.csv')), index=False)

    # --- Graph ---
    plt.figure(figsize=(12,6))
    plt.plot(df.index, df['Average Price'], label='Actual', color='blue')
    plt.plot(df.index, df['Predicted'], label='Predicted', color='red', linestyle='dashed')
    plt.plot(df.index, df['MA_7'], label='MA7', color='green', linestyle='--')
    plt.plot(df.index, df['MA_30'], label='MA30', color='orange', linestyle='--')
    plt.title(f"LSTM Daily Forecast - {file}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_graphs, file.replace('.csv','_lstm_graph.png')))
    plt.close()

    print(f"✅ {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={acc}%")
    gc.collect()

# ======================================================
# Save metrics
# ======================================================
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 Metrics saved to {metrics_file}")
print(metrics_df)


🚀 Processing: capsicum_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - loss: 0.0039 - mae: 0.0456 - val_loss: 0.0099 - val_mae: 0.0835 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0019 - mae: 0.0318 - val_loss: 0.0052 - val_mae: 0.0573 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.0014 - mae: 0.0273 - val_loss: 0.0066 - val_mae: 0.0672 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0011 - mae: 0.0247 - val_loss: 0.0093 - val_mae: 0.0847 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 9.7036e-04 - mae: 0.0228 - val_loss: 0.0058 - val_mae: 0.0640 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 9.1128e-04 - mae: 0.0223 - val_loss: 0.0084 - val_mae: 0.0817 - learning_rate: 0.0010
Epoch 7/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 7.8510e-04 - mae: 0.0201 - 

✅ capsicum_2010_2024_master.csv | MAE=137.73, RMSE=191.82, R²=0.9298, MAPE=5.07%, Accuracy=94.93%

🚀 Processing: onion_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - loss: 0.0044 - mae: 0.0395 - val_loss: 0.0012 - val_mae: 0.0241 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0013 - mae: 0.0232 - val_loss: 8.2268e-04 - val_mae: 0.0220 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 9.5540e-04 - mae: 0.0196 - val_loss: 7.8343e-04 - val_mae: 0.0194 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 8.2692e-04 - mae: 0.0182 - val_loss: 8.6811e-04 - val_mae: 0.0241 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 7.2780e-04 - mae: 0.0171 - val_loss: 5.9791e-04 - val_mae: 0.0179 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 6.3262e-04 - mae: 0.0160 - val_loss: 0.0012 - val_mae: 

✅ onion_2010_2024_master.csv | MAE=60.36, RMSE=80.01, R²=0.9859, MAPE=3.87%, Accuracy=96.13%

🚀 Processing: tomato_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - loss: 9.2702e-04 - mae: 0.0193 - val_loss: 0.0019 - val_mae: 0.0298 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 4.2605e-04 - mae: 0.0136 - val_loss: 0.0012 - val_mae: 0.0221 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 3.3745e-04 - mae: 0.0119 - val_loss: 0.0010 - val_mae: 0.0200 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 2.7603e-04 - mae: 0.0108 - val_loss: 0.0013 - val_mae: 0.0208 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 2.3983e-04 - mae: 0.0102 - val_loss: 0.0012 - val_mae: 0.0207 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 2.3509e-04 - mae: 0.0098 - val_loss: 0.0011 - val_mae: 0.0191 - lear

✅ tomato_2010_2024_master.csv | MAE=62.07, RMSE=104.32, R²=0.9819, MAPE=4.62%, Accuracy=95.38%

🚀 Processing: wheat_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - loss: 0.0037 - mae: 0.0373 - val_loss: 0.0042 - val_mae: 0.0554 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0011 - mae: 0.0227 - val_loss: 0.0113 - val_mae: 0.1005 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 8.5342e-04 - mae: 0.0194 - val_loss: 0.0065 - val_mae: 0.0725 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 7.1721e-04 - mae: 0.0178 - val_loss: 0.0084 - val_mae: 0.0844 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 5.9026e-04 - mae: 0.0158 - val_loss: 0.0098 - val_mae: 0.0930 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 5.3117e-04 - mae: 0.0149 - val_loss: 0.0155 - val_mae: 0.1192 - learning_ra

✅ wheat_2010_2024_master.csv | MAE=39.19, RMSE=67.06, R²=0.9823, MAPE=1.57%, Accuracy=98.43%

📊 Metrics saved to lstm_metrics.csv
                        Crop     MAE    RMSE    R2  MAPE(%)  Accuracy(%)
0  capsicum_2010_2024_master  137.73  191.82  0.93     5.07        94.93
1     onion_2010_2024_master   60.36   80.01  0.99     3.87        96.13
2    tomato_2010_2024_master   62.07  104.32  0.98     4.62        95.38
3     wheat_2010_2024_master   39.19   67.06  0.98     1.57        98.43


In [ ]:
#GRU

In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# ======================================================
# Output directories
# ======================================================
input_folder = "dataset"
output_models = "gru_output_models"
output_csv = "gru_output_csv"
output_graphs = "gru_output_graphs"
metrics_file = "gru_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# ======================================================
# Config
# ======================================================
look_back = 30
batch_size = 32
epochs = 60
val_split = 0.15
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# ======================================================
# Helper Functions
# ======================================================
def parse_dates_safe(date_series):
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def create_multivariate_dataset(y_scaled, exog_scaled, look_back):
    n = len(y_scaled)
    X, Y = [], []
    for i in range(n - look_back):
        seq_y = y_scaled[i:i+look_back, 0]
        seq_exog = exog_scaled[i:i+look_back, :]
        seq = np.concatenate([seq_y.reshape(-1,1), seq_exog], axis=1)
        X.append(seq)
        Y.append(y_scaled[i+look_back, 0])
    return np.array(X), np.array(Y)

# ======================================================
# Build GRU Model
# ======================================================
def build_gru_model(input_shape, units=64, dropout_rate=0.2, lr=0.001):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.GRU(units, return_sequences=True),
        layers.Dropout(dropout_rate),
        layers.GRU(units//2),
        layers.Dropout(dropout_rate),
        layers.Dense(units//2, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer=optimizers.Adam(learning_rate=lr), loss='mse', metrics=['mae'])
    return model

# ======================================================
# Main Loop
# ======================================================
metrics_list = []

for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue
    print(f"\n🚀 Processing: {file}")
    path = os.path.join(input_folder, file)
    df = pd.read_csv(path)

    # --- Handle Dates ---
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)

    # --- Fill and align data ---
    df['Average Price'] = df['Average Price'].ffill().bfill().fillna(df['Average Price'].mean())
    df = df.asfreq('D')
    df['Average Price'] = df['Average Price'].ffill().bfill()

    # --- Feature Engineering ---
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_7'] = df['Average Price'].shift(7)
    df['MA_7'] = df['Average Price'].rolling(window=7).mean()
    df['MA_30'] = df['Average Price'].rolling(window=30).mean()
    df = df.bfill()

    # --- Scaling ---
    y = df[['Average Price']].values.astype('float32')
    exog = df[['Lag_1', 'Lag_7', 'MA_7', 'MA_30']].values.astype('float32')

    y_scaler, exog_scaler = MinMaxScaler(), MinMaxScaler()
    y_scaled = y_scaler.fit_transform(y)
    exog_scaled = exog_scaler.fit_transform(exog)

    # --- Sequence Creation ---
    X_all, y_all = create_multivariate_dataset(y_scaled, exog_scaled, look_back)
    if len(X_all) == 0:
        continue

    # --- Train / Validation Split ---
    val_size = int(len(X_all) * val_split)
    X_train, y_train = X_all[:-val_size], y_all[:-val_size]
    X_val, y_val = X_all[-val_size:], y_all[-val_size:]

    # --- Build and Train ---
    model = build_gru_model((look_back, X_train.shape[2]))
    es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

    model.fit(X_train, y_train, validation_data=(X_val, y_val),
              epochs=epochs, batch_size=batch_size, callbacks=[es, rl], verbose=1)

    # --- Predictions ---
    preds_scaled = model.predict(X_all)
    preds = y_scaler.inverse_transform(preds_scaled).flatten()

    df['Predicted'] = np.nan
    df.iloc[look_back:, df.columns.get_loc('Predicted')] = preds
    df = df.round(2)

    # --- Metrics ---
    mask = ~np.isnan(df['Predicted'])
    y_true, y_pred = df.loc[mask, 'Average Price'], df.loc[mask, 'Predicted']

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    acc = round(100 - mape, 2)

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': acc
    })

    # --- Save Outputs ---
    model.save(os.path.join(output_models, file.replace('.csv','_gru_model.h5')))
    df_reset = df.reset_index()
    df_reset.to_csv(os.path.join(output_csv, file.replace('.csv','_gru_updated.csv')), index=False)

    # --- Graph ---
    plt.figure(figsize=(12,6))
    plt.plot(df.index, df['Average Price'], label='Actual', color='blue')
    plt.plot(df.index, df['Predicted'], label='Predicted', color='red', linestyle='dashed')
    plt.plot(df.index, df['MA_7'], label='MA7', color='green', linestyle='--')
    plt.plot(df.index, df['MA_30'], label='MA30', color='orange', linestyle='--')
    plt.title(f"GRU Daily Forecast - {file}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_graphs, file.replace('.csv','_gru_graph.png')))
    plt.close()

    print(f"✅ {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={acc}%")
    gc.collect()

# ======================================================
# Save metrics
# ======================================================
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 Metrics saved to {metrics_file}")
print(metrics_df)


🚀 Processing: capsicum_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 16s 38ms/step - loss: 0.0070 - mae: 0.0556 - val_loss: 0.0027 - val_mae: 0.0389 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 0.0019 - mae: 0.0319 - val_loss: 0.0025 - val_mae: 0.0386 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 0.0013 - mae: 0.0263 - val_loss: 0.0054 - val_mae: 0.0638 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 0.0011 - mae: 0.0239 - val_loss: 0.0054 - val_mae: 0.0648 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 9.7335e-04 - mae: 0.0223 - val_loss: 0.0034 - val_mae: 0.0486 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 8.1495e-04 - mae: 0.0206 - val_loss: 0.0055 - val_mae: 0.0661 - learning_rate: 0.0010
Epoch 7/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 6.8532e-04 - mae: 0.0187 - 

✅ capsicum_2010_2024_master.csv | MAE=96.42, RMSE=137.04, R²=0.9642, MAPE=3.6%, Accuracy=96.4%

🚀 Processing: onion_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - loss: 0.0020 - mae: 0.0291 - val_loss: 0.0010 - val_mae: 0.0238 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 9.6181e-04 - mae: 0.0204 - val_loss: 0.0016 - val_mae: 0.0335 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 6.3249e-04 - mae: 0.0165 - val_loss: 0.0033 - val_mae: 0.0490 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 5.3305e-04 - mae: 0.0153 - val_loss: 0.0034 - val_mae: 0.0466 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 4.6053e-04 - mae: 0.0142 - val_loss: 0.0040 - val_mae: 0.0539 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 4.1975e-04 - mae: 0.0131 - val_loss: 0.0026 - val_mae: 0.0421 - learni

✅ onion_2010_2024_master.csv | MAE=49.51, RMSE=78.92, R²=0.9862, MAPE=2.95%, Accuracy=97.05%

🚀 Processing: tomato_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - loss: 0.0017 - mae: 0.0245 - val_loss: 0.0013 - val_mae: 0.0233 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 3.4206e-04 - mae: 0.0120 - val_loss: 9.0270e-04 - val_mae: 0.0195 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 2.5514e-04 - mae: 0.0100 - val_loss: 5.7453e-04 - val_mae: 0.0150 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 2.4114e-04 - mae: 0.0098 - val_loss: 5.2215e-04 - val_mae: 0.0145 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 1.8944e-04 - mae: 0.0091 - val_loss: 6.0703e-04 - val_mae: 0.0147 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 1.7144e-04 - mae: 0.0084 - val_loss: 0.0014 - val_mae: 

✅ tomato_2010_2024_master.csv | MAE=39.28, RMSE=67.27, R²=0.9925, MAPE=3.03%, Accuracy=96.97%

🚀 Processing: wheat_2010_2024_master.csv
Epoch 1/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - loss: 0.0027 - mae: 0.0334 - val_loss: 0.0075 - val_mae: 0.0808 - learning_rate: 0.0010
Epoch 2/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 0.0011 - mae: 0.0217 - val_loss: 0.0147 - val_mae: 0.1164 - learning_rate: 0.0010
Epoch 3/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 7.7093e-04 - mae: 0.0185 - val_loss: 0.0218 - val_mae: 0.1431 - learning_rate: 0.0010
Epoch 4/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 6.7351e-04 - mae: 0.0171 - val_loss: 0.0203 - val_mae: 0.1374 - learning_rate: 0.0010
Epoch 5/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 5.7909e-04 - mae: 0.0161 - val_loss: 0.0150 - val_mae: 0.1170 - learning_rate: 0.0010
Epoch 6/60
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 5.0158e-04 - mae: 0.0143 - val_loss: 0.0159 - val_mae: 0.1205 - learning_ra

✅ wheat_2010_2024_master.csv | MAE=69.9, RMSE=95.97, R²=0.9638, MAPE=2.84%, Accuracy=97.16%

📊 Metrics saved to gru_metrics.csv
                        Crop    MAE    RMSE    R2  MAPE(%)  Accuracy(%)
0  capsicum_2010_2024_master  96.42  137.04  0.96     3.60        96.40
1     onion_2010_2024_master  49.51   78.92  0.99     2.95        97.05
2    tomato_2010_2024_master  39.28   67.27  0.99     3.03        96.97
3     wheat_2010_2024_master  69.90   95.97  0.96     2.84        97.16
